<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
!pip install - q pymatgen

     |████████████████████████████████| 40.6 MB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 98 kB 7.2 MB/s 
     |████████████████████████████████| 292 kB 36.3 MB/s 
     |████████████████████████████████| 38.1 MB 53.9 MB/s 
     |████████████████████████████████| 109 kB 65.4 MB/s 
     |████████████████████████████████| 546 kB 64.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.analysis.local_env import CrystalNN, JmolNN
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import networkx as nx
import os
import json

plt.style.use('seaborn-white')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
private = "1OFmiprkJwvwWL9MnCxGFvp1l_IDwI79x"
public  = "1f2I_YMV0NY8bFw2jZN4TIekNcx8KLbOG"

In [ ]:
gdd.download_file_from_google_drive(file_id=private, 
                                    dest_path='./private.tar.gz', 
                                    unzip=False)

gdd.download_file_from_google_drive(file_id=public, 
                                    dest_path='./public.tar.gz', 
                                    unzip=False)


!tar -xzvf  private.tar.gz >/dev/null 2>&1
!tar -xzvf  public.tar.gz >/dev/null 2>&1

In [ ]:
private = "dichalcogenides_private/"
public  = "dichalcogenides_public/"

targets = pd.read_csv(public+'targets.csv')

d_y = dict(zip(targets['_id'].values.tolist(), 
               targets['band_gap'].values.tolist()))

In [ ]:
def read_pymaten_structure(path_to_file):
    with open(path_to_file, "r") as f:
        d = json.load(f)
        structure = Structure.from_dict(d)
    return structure

In [ ]:
files_public = list(os.listdir(public + 'structures/'))
files_private = list(os.listdir(private + 'structures/'))

In [ ]:
nn = JmolNN()

In [ ]:
def rmjac(s):
    return s.replace('.json', '.csv')

In [ ]:
private_dir = 'private_JmolNN/'
public_dir = 'public_JmolNN/'

if not os.path.exists(private_dir):
    os.makedirs(private_dir)

if not os.path.exists(public_dir):
    os.makedirs(public_dir)

In [ ]:
def structure_to_adjacency(files, directroy_from, directory_to):
    for file in tqdm(files):
        if rmjac(file) not in os.listdir(directory_to):
            st = read_pymaten_structure(directroy_from + 'structures/' + file)

            graph = nn.get_bonded_structure(st, weights=True)
            G = graph.graph

            G = nx.to_undirected(G)

            df_G = nx.to_pandas_adjacency(G)
            df_G.to_csv(f'{directory_to}/{rmjac(file)}', index=False)

In [ ]:
structure_to_adjacency(files_public, public, public_dir)

  0%|          | 0/2966 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
!zip -r -q public_CrystalNN.zip public_JmolNN/ 

In [ ]:
structure_to_adjacency(files_private, private, private_dir)

In [ ]:
!zip -r -q private_JmolNN.zip private_JmolNN/ 